In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr, spearmanr

import env
import wrangle
import sklearn.preprocessing
from sklearn.linear_model import LinearRegression
import sklearn.preprocessing
from sklearn.metrics import explained_variance_score
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression


In [2]:
df = wrangle.get_zillow_data()

OperationalError: (pymysql.err.OperationalError) (2013, 'Lost connection to MySQL server during query ([Errno 54] Connection reset by peer)')
[SQL: 
        SELECT properties_2017.bedroomcnt AS Number_of_Bedrooms,
        properties_2017.bathroomcnt AS Number_of_Bathrooms,
        properties_2017.calculatedfinishedsquarefeet AS Square_Feet, 
        properties_2017.taxvaluedollarcnt AS Tax_Appraised_Value, 
        properties_2017.yearbuilt AS Year_Built, 
        properties_2017.taxamount AS Tax_Assessed, properties_2017.fips AS County_Code,
        properties_2017.lotsizesquarefeet AS Lot_Size
        FROM properties_2017
        JOIN propertylandusetype USING (propertylandusetypeid)
        WHERE propertylandusedesc = 'Single Family Residential';
        ]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
df = wrangle.prepare_zillow_data(df)

In [ ]:
df.head(1)

## Exploration

In [ ]:
df.describe().T

In [ ]:
train,validate,test = wrangle.split_zillow_data(df)

### Create X and Y train, validate, test

In [ ]:
X_train = train.drop(columns =['Tax_Appraised_Value','Year_Built','Tax_Assessed' ,'County','Lot_Size','County_Los Angeles','County_Orange','County_Ventura','Tax_Rate'])
y_train = train['Tax_Appraised_Value']

X_validate = validate.drop(columns =['Tax_Appraised_Value','Year_Built','Tax_Assessed' ,'County','Lot_Size','County_Los Angeles','County_Orange','County_Ventura','Tax_Rate'])
y_validate = validate['Tax_Appraised_Value']

X_test = test.drop(columns =['Tax_Appraised_Value','Year_Built','Tax_Assessed' ,'County','Lot_Size','County_Los Angeles','County_Orange','County_Ventura','Tax_Rate'])
y_test = test['Tax_Appraised_Value']

### OLS Modeling - without scaling
 **Bed, Bath, Square Feet Only = RMSE = 559573**

In [ ]:
y_train = pd.DataFrame(y_train)
y_validate = pd.DataFrame(y_validate)

In [ ]:
# create the model
lm = LinearRegression(normalize=True)

In [ ]:
# fit the model
lm.fit(X_train, y_train.Tax_Appraised_Value)

In [ ]:
# train the model
y_train['TAV_pred_lm'] = lm.predict(X_train)

In [ ]:
# evaluate RMSE
rmse_train = mean_squared_error(y_train.Tax_Appraised_Value, y_train.TAV_pred_lm)**(1/2)
print('RMSE in Sample: ', round(rmse_train,2))

In [ ]:
R2

In [ ]:
X_train1 = train.drop(columns =['Tax_Appraised_Value','Year_Built','Tax_Assessed' ,'Tax_Rate'])
y_train1 = train['Tax_Appraised_Value']

X_validate1 = validate.drop(columns =['Tax_Appraised_Value','Year_Built','Tax_Assessed' ,'Tax_Rate'])
y_validate1 = validate['Tax_Appraised_Value']

X_test1 = test.drop(columns =['Tax_Appraised_Value','Year_Built','Tax_Assessed' ,'Tax_Rate'])
y_test1 = test['Tax_Appraised_Value']

In [ ]:
y_train1 = pd.DataFrame(y_train1)
y_validate1 = pd.DataFrame(y_validate1)

In [ ]:
# create the model
lm = LinearRegression(normalize=True)

In [ ]:
# fit the model
lm.fit(X_train1, y_train1.Tax_Appraised_Value)

In [ ]:
# train the model
y_train1['TAV_pred_lm1'] = lm.predict(X_train1)

In [ ]:
# evaluate RMSE
rmse_train1 = mean_squared_error(y_train1.Tax_Appraised_Value, y_train1.TAV_pred_lm1)**(1/2)
print('RMSE in Sample: ', round(rmse_train1,2))

### OLS Modeling - with scaling

### Scale using standard scaler

In [ ]:
# scaler = sklearn.preprocessing.StandardScaler()
# scaler = sklearn.preprocessing.QuantileTransformer(output_distribution='normal')
# scaler = sklearn.preprocessing.MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

In [ ]:
y_train = pd.DataFrame(y_train)
y_validate = pd.DataFrame(y_validate)

In [ ]:
# create the model
lm = LinearRegression(normalize=True)

In [ ]:
# fit the model
lm.fit(X_train_scaled, y_train.Tax_Appraised_Value)

In [ ]:
# train the model
y_train['TAV_pred_lm_scaled'] = lm.predict(X_train_scaled)

In [ ]:
# evaluate RMSE
rmse_train_scaled = mean_squared_error(y_train.Tax_Appraised_Value, y_train.TAV_pred_lm_scaled)**(1/2)
print('RMSE in Sample- Standard Scaler: ', round(rmse_train_scaled,2))

### Feature Engineering 

In [ ]:
f_selector = SelectKBest(f_regression, k=3)


f_selector.fit(X_train_scaled, y_train)

# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

In [ ]:
feature_mask